# Init

In [ ]:
!gdown 1s1WZkcd42vU-8hxHMtwVoYjeGD6_PQNA
!gdown 1s5He5p-vl0HGt8qLPVNiplcE0g_vQ8iQ

import nltk
nltk.download('punkt')
nltk.download('reuters')
import pandas as pd
from nltk import ngrams, word_tokenize, Counter, edit_distance
from nltk.metrics.distance import edit_distance
from IPython.display import display
from nltk.tokenize.treebank import TreebankWordDetokenizer

Downloading...
From: https://drive.google.com/uc?id=1s1WZkcd42vU-8hxHMtwVoYjeGD6_PQNA
To: /content/test.txt
100% 3.20k/3.20k [00:00<00:00, 11.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1s5He5p-vl0HGt8qLPVNiplcE0g_vQ8iQ
To: /content/answer.txt
100% 3.22k/3.22k [00:00<00:00, 13.1MB/s]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package reuters to /root/nltk_data...


# Edit Distance

In [ ]:
# Kode dari artikel Peter Norvig: https://norvig.com/spell-correct.html
# Intinya generate semua string yang jaraknya distance tertentu dari word
# Pake yang namanya Damerau-Levenshtein distance.
# Ada transpose sebagai tambahan dari operasi lev biasa.
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "All edits that are two edits away from `word`."
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1))

# Model

In [ ]:
# Bikin class biar rapih
class ngram_checker:
    # Kasih text training sama angka n_gram yang diinginkan.
    def __init__(self, text:int, n_gram:int):
        self.n_gram = n_gram
        self.word_freq = self._get_freq(text, n_gram)
        self.vocab = set(self.word_freq["word"].apply(lambda x: x[0]).unique())

    # Generate df ngram, dengan padding
    def _get_freq(self, text:str, n_gram:int):
        token = word_tokenize(text)
        grams = ngrams(token, n_gram, pad_left = True, pad_right = True, left_pad_symbol="<s>", right_pad_symbol="</s>")
        df = pd.DataFrame(Counter(grams).items(), columns=["word", "freq"])
        return df

    # Dari sebuah list of tokens dimana elemen terakhirnya typo,
    # buat daftar frekuensi huruf2 yang jaraknya 2 edit distance
    # aka token pengganti.
    def prob_grams(self, tokens:list[str]):
        ret = pd.DataFrame(columns=self.word_freq.columns)

        # Atur padding
        if len(tokens) < self.n_gram:
            tokens = (["<s>"] * (self.n_gram - len(tokens))) + tokens
        else:
            tokens = tokens[-self.n_gram:]

        # Ret kalo ada di vocab, bukan typo.
        word = tokens[-1]
        if word in self.vocab:
            return ret

        # Generate semua string yang mungkin buat edit distance 2, terus pilih yang dalam vocab kita.
        possible_edits = edits2(word)
        vocab_words = possible_edits.intersection(self.vocab)

        # filter dulu yang depannya sesuai, biar kodenya lebih cepet
        freq = self.word_freq[self.word_freq["word"].apply(lambda x: x[:-1] == tuple(tokens[:-1]))]

        # sekarang filter yang belakangnya sesuai
        for word in vocab_words:
            ret = pd.concat((ret, freq[freq["word"].apply(lambda x: x[-1] == word)]))

        return ret.sort_values("freq", ascending=False)

    # Fungsi yang bakal ngejalanin prob_grams secara otomatis.
    # Bakal nerima string habis itu secara otomatis tokenize.
    # Habis ditokenize diproses pake fungsi atas buat dapet list2 token pengganti.
    # Ambil token pengganti dengan probabilitas tertinggi, terus sambung.
    def autocorrect(self, text:str):
        # Hasil autocorrect
        new_tokens = []

        # Hubungin kata yang satu dengan yang lain
        tokens = word_tokenize(text)
        ngram_tokens = list(ngrams(tokens, self.n_gram, pad_left=True, left_pad_symbol = "<s>"))
        # Bikin list of list biar bisa dimutate
        ngram_tokens = [list(l) for l in ngram_tokens]

        # Jalanin
        for t in ngram_tokens:
            result = self.prob_grams(t)
            if len(result) == 0:
                new_tokens.append(t[-1])
            else:
                print(f"Corrected {t} to: ", end="")
                t[-1] = result.iloc[0]["word"][-1]
                new_tokens.append(t[-1])
                print(f"{t}")
        return TreebankWordDetokenizer().detokenize(new_tokens)

# Training
Pake dataset reuters.

In [ ]:
from nltk.corpus import reuters

In [ ]:
text = reuters.raw()

In [ ]:
print(f"Training using {len(word_tokenize(text))} tokens")
text[:200]

Training using 1548214 tokens


"ASIAN EXPORTERS FEAR DAMAGE FROM U.S.-JAPAN RIFT\n  Mounting trade friction between the\n  U.S. And Japan has raised fears among many of Asia's exporting\n  nations that the row could inflict far-reachin"

In [ ]:
unigram = ngram_checker(text, 1)
bigram = ngram_checker(text, 2)
trigram = ngram_checker(text, 3)
fourgram = ngram_checker(text, 4)

In [ ]:
print(f"Unigram length: {len(unigram.word_freq)}")
display(unigram.word_freq.sort_values("freq", ascending=False).head(3))
print(f"Bigram length: {len(bigram.word_freq)}")
display(bigram.word_freq.sort_values("freq", ascending=False).head(3))
print(f"Trigram length: {len(trigram.word_freq)}")
display(trigram.word_freq.sort_values("freq", ascending=False).head(3))
print(f"Fourgram length: {len(fourgram.word_freq)}")
display(fourgram.word_freq.sort_values("freq", ascending=False).head(3))

Unigram length: 63318


,word,freq
11,"(the,)",58220
32,"(,,)",53665
37,"(.,)",50901


Bigram length: 441155


,word,freq
192,"(lt, ;)",8694
191,"(&, lt)",8694
91,"(., The)",8635


Trigram length: 929652


,word,freq
198,"(&, lt, ;)",8694
570,"(said, ., The)",1516
569,"(he, said, .)",1230


Fourgram length: 1216597


,word,freq
13162,"(INC, &, lt, ;)",1059
586,"(,, '', he, said)",734
3552,"(., The, company, said)",732


# Testing

## Simple sentences

In [ ]:
typo_string = "at the end otf"
tokens = word_tokenize(typo_string)

display(unigram.prob_grams(tokens).head(5))
display(bigram.prob_grams(tokens).head(5))
display(trigram.prob_grams(tokens).head(5))
display(fourgram.prob_grams(tokens).head(5))

,word,freq
20,"(of,)",35949
53,"(to,)",33981
76,"(it,)",8834
135,"(lt,)",8694
55,"(on,)",8412


,word,freq
2646,"(end, of)",548
20261,"(end, to)",25
40318,"(end, on)",7
76003,"(end, its)",3
37935,"(end, at)",3


,word,freq
3067,"(the, end, of)",501


,word,freq
3166,"(at, the, end, of)",250


## Long texts

In [ ]:
f = open("test.txt")
test = f.read()
f.close()
test

'In a groundbreaking study, researchers at the renowned Lfie Sciences Institute have made a significant btreakthrough in the field of medical traeatment, offering new hope for patietns around the world. The study, led by Dr. Alexnder Johnson, focused on developing a novev therapy for neurologiucal disorders, particularly Alzheimer\'s diseaese.\n\nThe results of the study were publisehd today in the prestiogious jorunal, Science Advances, sending ripples of excitement thourghout the meducal community. The research team, comprizing experts from various diciplines, employed a combinatuion of gene therapy and stem cell research to develop a proimising new approach.\n\nDr. Johnson expalined, "Our study focused on enhancing neuronal regneration by tarhgting specific genetic markeers associated with neurodegenerative conditins. We utlilized advanced CRISPR techbology to modify and activate these markees, which ultimately led to a signifcant improement in cognitive functions in our animal mode

In [ ]:
unigram_res = unigram.autocorrect(test)

Corrected ['renowned'] to: ['renewed']
Corrected ['Lfie'] to: ['five']
Corrected ['btreakthrough'] to: ['breakthrough']
Corrected ['traeatment'] to: ['treatment']
Corrected ['patietns'] to: ['patents']
Corrected ['Alexnder'] to: ['lender']
Corrected ['novev'] to: ['over']
Corrected ['diseaese'] to: ['disease']
Corrected ['publisehd'] to: ['published']
Corrected ['jorunal'] to: ['journal']
Corrected ['Advances'] to: ['advance']
Corrected ['ripples'] to: ['apples']
Corrected ['thourghout'] to: ['throughout']
Corrected ['meducal'] to: ['medical']
Corrected ['comprizing'] to: ['comprising']
Corrected ['diciplines'] to: ['discipline']
Corrected ['combinatuion'] to: ['combination']
Corrected ['gene'] to: ['were']
Corrected ['proimising'] to: ['promising']
Corrected ['expalined'] to: ['explained']
Corrected ['regneration'] to: ['generation']
Corrected ['tarhgting'] to: ['targeting']
Corrected ['markeers'] to: ['markets']
Corrected ['conditins'] to: ['conditions']
Corrected ['utlilized'] to: [

In [ ]:
bigram_res = bigram.autocorrect(test)

Corrected ['the', 'renowned'] to: ['the', 'renewed']
Corrected ['for', 'patietns'] to: ['for', 'patents']
Corrected ['a', 'novev'] to: ['a', 'move']
Corrected ['the', 'meducal'] to: ['the', 'medical']
Corrected [',', 'comprizing'] to: [',', 'comprising']
Corrected ['a', 'combinatuion'] to: ['a', 'combination']
Corrected ['of', 'gene'] to: ['of', 'one']
Corrected ['a', 'proimising'] to: ['a', 'promising']
Corrected ['these', 'markees'] to: ['these', 'markets']
Corrected ['a', 'signifcant'] to: ['a', 'significant']
Corrected ['are', 'excting'] to: ['are', 'acting']
Corrected ['further', 'researc'] to: ['further', 'research']
Corrected ['clinical', 'triasl'] to: ['clinical', 'trials']
Corrected ['a', 'ray'] to: ['a', 'day']
Corrected ['is', 'imporant'] to: ['is', 'important']
Corrected ['with', 'cautin'] to: ['with', 'caution']
Corrected ['to', 'trnslate'] to: ['to', 'translate']
Corrected ['.', 'Additoinally'] to: ['.', 'Additionally']
Corrected ['the', 'studt'] to: ['the', 'start']
Corr

In [ ]:
trigram_res = trigram.autocorrect(test)

Corrected ['to', 'a', 'signifcant'] to: ['to', 'a', 'significant']
Corrected ['it', 'is', 'imporant'] to: ['it', 'is', 'important']
Corrected [',', 'the', 'studt'] to: [',', 'the', 'start']
Corrected [',', 'a', 'prominnet'] to: [',', 'a', 'prominent']
Corrected ['at', 'the', 'Univeristy'] to: ['at', 'the', 'University']
Corrected ['have', 'already', 'srtated'] to: ['have', 'already', 'started']
Corrected ['of', 'the', 'stud'] to: ['of', 'the', 'study']
Corrected ['of', 'the', 'scietific'] to: ['of', 'the', 'scientific']


In [ ]:
fourgram_res = fourgram.autocorrect(test)

Corrected ['led', 'to', 'a', 'signifcant'] to: ['led', 'to', 'a', 'significant']
Corrected [',', 'it', 'is', 'imporant'] to: [',', 'it', 'is', 'important']


In [ ]:
# Kalo trigram ketemu, pake punya trigram, kalo gak pake punya bigram, kalo ga unigram.
class fallback_ngram:
    def __init__(self, models):
        self.models = models


    def autocorrect(self, text:str):
        new_tokens = []

        # Hubungin kata yang satu dengan yang lain
        tokens = word_tokenize(text)
        ngram_tokens = list(ngrams(tokens, 3, pad_left=True, left_pad_symbol = "<s>"))
        # Bikin list of list biar bisa dimutate
        ngram_tokens = [list(l) for l in ngram_tokens]

        # Jalanin
        for t in ngram_tokens:
            found = False
            for model in self.models:
                t = t[-model.n_gram:]
                result = model.prob_grams(t)
                if len(result):
                    print(f"Corrected {t} to: ", end="")
                    t[-1] = result.iloc[0]["word"][-1]
                    new_tokens.append(t[-1])
                    print(f"{t} ({model.n_gram}-gram)")
                    found = True
                    break
            if not found:
                new_tokens.append(t[-1])
        return TreebankWordDetokenizer().detokenize(new_tokens)

In [ ]:
combined_model = fallback_ngram([fourgram, trigram, bigram, unigram])
combined_res = combined_model.autocorrect(test)

Corrected ['the', 'renowned'] to: ['the', 'renewed'] (2-gram)
Corrected ['Lfie'] to: ['five'] (1-gram)
Corrected ['btreakthrough'] to: ['breakthrough'] (1-gram)
Corrected ['traeatment'] to: ['treatment'] (1-gram)
Corrected ['for', 'patietns'] to: ['for', 'patents'] (2-gram)
Corrected ['Alexnder'] to: ['lender'] (1-gram)
Corrected ['a', 'novev'] to: ['a', 'move'] (2-gram)
Corrected ['diseaese'] to: ['disease'] (1-gram)
Corrected ['publisehd'] to: ['published'] (1-gram)
Corrected ['jorunal'] to: ['journal'] (1-gram)
Corrected ['Advances'] to: ['advance'] (1-gram)
Corrected ['ripples'] to: ['apples'] (1-gram)
Corrected ['thourghout'] to: ['throughout'] (1-gram)
Corrected ['the', 'meducal'] to: ['the', 'medical'] (2-gram)
Corrected [',', 'comprizing'] to: [',', 'comprising'] (2-gram)
Corrected ['diciplines'] to: ['discipline'] (1-gram)
Corrected ['a', 'combinatuion'] to: ['a', 'combination'] (2-gram)
Corrected ['of', 'gene'] to: ['of', 'one'] (2-gram)
Corrected ['a', 'proimising'] to: ['a'

In [ ]:
combined_model2 = fallback_ngram([bigram, unigram])
combined_res2 = combined_model2.autocorrect(test)

Corrected ['the', 'renowned'] to: ['the', 'renewed'] (2-gram)
Corrected ['Lfie'] to: ['five'] (1-gram)
Corrected ['btreakthrough'] to: ['breakthrough'] (1-gram)
Corrected ['traeatment'] to: ['treatment'] (1-gram)
Corrected ['for', 'patietns'] to: ['for', 'patents'] (2-gram)
Corrected ['Alexnder'] to: ['lender'] (1-gram)
Corrected ['a', 'novev'] to: ['a', 'move'] (2-gram)
Corrected ['diseaese'] to: ['disease'] (1-gram)
Corrected ['publisehd'] to: ['published'] (1-gram)
Corrected ['jorunal'] to: ['journal'] (1-gram)
Corrected ['Advances'] to: ['advance'] (1-gram)
Corrected ['ripples'] to: ['apples'] (1-gram)
Corrected ['thourghout'] to: ['throughout'] (1-gram)
Corrected ['the', 'meducal'] to: ['the', 'medical'] (2-gram)
Corrected [',', 'comprizing'] to: [',', 'comprising'] (2-gram)
Corrected ['diciplines'] to: ['discipline'] (1-gram)
Corrected ['a', 'combinatuion'] to: ['a', 'combination'] (2-gram)
Corrected ['of', 'gene'] to: ['of', 'one'] (2-gram)
Corrected ['a', 'proimising'] to: ['a'

# Evaluation

In [ ]:
f = open("answer.txt")
ans = f.read()
f.close()
ans_token = word_tokenize(ans)

In [ ]:
for result, name in [(test, "Original Text"), (unigram_res, "Unigram"), (bigram_res, "Bigram"), (trigram_res, "Trigram"), (fourgram_res, "Fourgram"), (combined_res, "1-4 Gram"), (combined_res2, "1-2 Gram")]:
    tokens = word_tokenize(result)
    mistakes = sum([a == b for (a, b) in zip(tokens, ans_token)])
    print(f"Correctly spelt words for {name} is {mistakes} / {len(tokens)}. {mistakes/len(tokens)}")

# Buat Screenshot PPT

In [ ]:
print("renowned")
display(unigram.word_freq[unigram.word_freq["word"] == tuple(["renowned"])])
print("\n\nrenewed")
display(unigram.word_freq[unigram.word_freq["word"] == tuple(["renewed"])])

In [ ]:
bigram = ngram_checker(text, 2)

In [ ]:
bigram_test = bigram.word_freq
bigram_test["word1"] = bigram_test["word"].apply(lambda x: x[0])
bigram_test["word2"] = bigram_test["word"].apply(lambda x: x[1])
bigram_test = bigram_test.drop("word", axis=1)
words =["for", "the", "vs"]
bigram_test = bigram_test[bigram_test["word1"].isin(words) & bigram_test["word2"].isin(words)]
display(bigram_test)
display(bigram_test.pivot_table(values="freq", index="word1", columns="word2", fill_value=0))

In [ ]:
combined_res2